import pandas as pd
import numpy as np
# ^^^ pyforest auto-imports - don't write above this line
Notebook to create the final list of subjects. Criteria:
- Have reactivity information.
- both stroop, msit and rest have full atlas coverage.
- be low motion for both tasks.
- something else? resting as well?

In [1]:
import numpy as np
import pandas as pd
import regex as re
import os 
from tqdm import tqdm
from nilearn import image, input_data
from joblib import Parallel, delayed

In [2]:
# No nans in this dataset for reactivity
pd.read_csv("../data/demo/pip_reactivity_2020.csv").filter(regex="reactivity").isna().any()

sbp_reactivity_both      False
sbp_reactivity_stroop    False
sbp_reactivity_msit      False
dbp_reactivity_both      False
dbp_reactivity_stroop    False
dbp_reactivity_msit      False
map_reactivity_both      False
map_reactivity_stroop    False
map_reactivity_msit      False
hr_reactivity_both       False
hr_reactivity_stroop     False
hr_reactivity_msit       False
dtype: bool

In [3]:
# subjects with reactivity
subjects_reactivity = pd.read_csv("../data/demo/pip_reactivity_2020.csv").id.to_list()

reactivity_df = pd.DataFrame({'SUB_ID': subjects_reactivity})
reactivity_df.head()

,SUB_ID
0,1002
1,1003
2,1004
3,1005
4,1006


In [23]:
pd.merge(pd.DataFrame({'ePrime.id':np.loadtxt("../data/subjects_intersect_motion_035.txt")}),
         pd.read_spss("../data/demo/PIP_n330_03_26_2019.sav"), on = 'ePrime.id').age.std()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

6.226693797482864

In [19]:
pd.merge(pd.DataFrame({'ePrime.id':np.loadtxt("../data/subjects_intersect_motion_035.txt")}),
         pd.read_spss("../data/demo/PIP_n330_03_26_2019.sav"), on = 'ePrime.id').gender.value_counts()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

MALE      123
FEMALE    119
Name: gender, dtype: int64

In [13]:
atlas_file = "../data/atlases/shen_2mm_268_parcellation.nii.gz"

def check_acquisition_task(img_file):
    """
    
    function to check that we have a full acquisition (280 scans) and 
    that atlas fit subject (all ROI have signal)
    
    """
    
    masker = input_data.NiftiLabelsMasker(atlas_file)
    roi_data = masker.fit_transform(img_file)
    n_obs = roi_data.shape[0]
    
    full_acq = False
    if n_obs == 280:
        full_acq = True
    
    roi_data_std = np.std(roi_data, axis=0)
    all_rois = ~np.any(roi_data_std == 0)
    is_ok = (full_acq==True) & (all_rois==True)
    
    return is_ok

def check_acquisition_rest(img_file):
    """
    
    function to check that we have a full acquisition (280 scans) and 
    that nifti masker 
    
    """
    
    masker = input_data.NiftiLabelsMasker(atlas_file)
    roi_data = masker.fit_transform(img_file)
    n_obs = roi_data.shape[0]
    
    full_acq = False
    if n_obs == 150:
        full_acq = True
    
    roi_data_std = np.std(roi_data, axis=0)
    all_rois = ~np.any(roi_data_std == 0)
    
    is_ok = (full_acq==True) & (all_rois==True)
    
    return is_ok

In [5]:
task_id = "stroop"
pattern = "sub-(.*)_ses-01_task-%s_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz" % task_id
pattern_confounders = "../data/confounders/task-%s" % task_id + "/sub-%s" + "_ses-01_task-%s_desc-confounds_regressors.tsv" % task_id
list_subjects_ids = []
list_subjects_fwd = []

for filename in tqdm(os.listdir("../data/preproc_bold/task-%s" % task_id)):
    subj = re.findall(pattern=pattern, string=filename)[0]

    list_subjects_ids.append(int(subj))
    
    fwd = pd.read_csv(pattern_confounders % subj, sep="\t").framewise_displacement.dropna().mean()
    list_subjects_fwd.append(fwd)
    
stroop_df = pd.DataFrame({'SUB_ID': list_subjects_ids, 'FWD': list_subjects_fwd})
stroop_df.head()

parallel = Parallel(n_jobs=-1)
is_ok = parallel(delayed(check_acquisition_task)("../data/preproc_bold/task-%s" % task_id + "/" + filename) \
                     for filename in tqdm(os.listdir("../data/preproc_bold/task-%s" % task_id)))
is_ok = np.array(is_ok)

stroop_df = stroop_df.loc[is_ok,:]

100%|██████████| 333/333 [03:52<00:00,  1.43it/s]


In [6]:
stroop_df.shape

(322, 2)

In [7]:
task_id = "msit"
pattern = "sub-(.*)_ses-01_task-%s_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz" % task_id
pattern_confounders = "../data/confounders/task-%s" % task_id + "/sub-%s" + "_ses-01_task-%s_desc-confounds_regressors.tsv" % task_id
list_subjects_ids = []
list_subjects_fwd = []

for filename in tqdm(os.listdir("../data/preproc_bold/task-%s" % task_id)):
    subj = re.findall(pattern=pattern, string=filename)[0]

    list_subjects_ids.append(int(subj))
    
    fwd = pd.read_csv(pattern_confounders % subj, sep="\t").framewise_displacement.dropna().mean()
    list_subjects_fwd.append(fwd)
    
msit_df = pd.DataFrame({'SUB_ID': list_subjects_ids, 'FWD': list_subjects_fwd})
msit_df.head()

parallel = Parallel(n_jobs=-1)
is_ok = parallel(delayed(check_acquisition_task)("../data/preproc_bold/task-%s" % task_id + "/" + filename) \
                     for filename in tqdm(os.listdir("../data/preproc_bold/task-%s" % task_id)))
is_ok = np.array(is_ok)

msit_df = msit_df.loc[is_ok,:]

100%|██████████| 335/335 [03:25<00:00,  1.63it/s]


In [8]:
msit_df.shape

(317, 2)

In [15]:
task_id = "rest"
pattern = "sub-(.*)_ses-01_task-%s_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz" % task_id
pattern_confounders = "../data/confounders/task-%s" % task_id + "/sub-%s" + "_ses-01_task-%s_desc-confounds_regressors.tsv" % task_id
list_subjects_ids = []
list_subjects_fwd = []

for filename in tqdm(os.listdir("../data/preproc_bold/task-%s" % task_id)):
    subj = re.findall(pattern=pattern, string=filename)[0]

    list_subjects_ids.append(int(subj))
    
    fwd = pd.read_csv(pattern_confounders % subj, sep="\t").framewise_displacement.dropna().mean()
    list_subjects_fwd.append(fwd)
    
resting_df = pd.DataFrame({'SUB_ID': list_subjects_ids, 'FWD': list_subjects_fwd})
resting_df.head()

parallel = Parallel(n_jobs=-1)
is_ok = parallel(delayed(check_acquisition_rest)("../data/preproc_bold/task-%s" % task_id + "/" + filename) \
                     for filename in tqdm(os.listdir("../data/preproc_bold/task-%s" % task_id)))
is_ok = np.array(is_ok)

resting_df = resting_df.loc[is_ok,:]

100%|██████████| 334/334 [01:56<00:00,  2.86it/s]


In [16]:
resting_df.shape

(309, 2)

Find intersection amonng subjects and save it to disk for further use for several thresholds

In [18]:
for motion_thr in [0.3, 0.35, 0.4]:
    stroop_low_motion_df = stroop_df[stroop_df.FWD < motion_thr]
    msit_low_motion_df = msit_df[msit_df.FWD < motion_thr]
    resting_low_motion_df = resting_df[resting_df.FWD < motion_thr]

    subjects_intersect = list(set(reactivity_df.SUB_ID.to_list()) & \
                               set(stroop_low_motion_df.SUB_ID.to_list()) & \
                               set(msit_low_motion_df.SUB_ID.to_list()) & \
                               set(resting_low_motion_df.SUB_ID.to_list())
                              )
    
    print(len(subjects_intersect))

    np.savetxt("../data/subjects_intersect_motion_%s.txt" % str(motion_thr*100).split(".")[0].zfill(3),  
               subjects_intersect, fmt="%d")

213
242
249


In [19]:
# The same but with a more stringet threshold on resting (0.2)
for motion_thr in [0.3, 0.35, 0.4]:
    stroop_low_motion_df = stroop_df[stroop_df.FWD < motion_thr]
    msit_low_motion_df = msit_df[msit_df.FWD < motion_thr]
    resting_low_motion_df = resting_df[resting_df.FWD < 0.2]

    subjects_intersect = list(set(reactivity_df.SUB_ID.to_list()) & \
                               set(stroop_low_motion_df.SUB_ID.to_list()) & \
                               set(msit_low_motion_df.SUB_ID.to_list()) & \
                               set(resting_low_motion_df.SUB_ID.to_list())
                              )
    print(len(subjects_intersect))
    np.savetxt("../data/subjects_intersect_motion_%s_rest_020.txt" % str(motion_thr*100).split(".")[0].zfill(3),  
               subjects_intersect, fmt="%d")

177
184
187
